In [1]:
from pyspark.sql.functions import *
import time
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
# df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2015/part-00000-300232b0-d488-4017-8741-efda0762a4d3-c000.tdv.gz", sep=r'\t', header=True, inferSchema="true")
df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2015/", sep=r'\t', header=True, inferSchema="true")

# df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2016/", sep=r'\t', header=True, inferSchema="true")
# df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2017/", sep=r'\t', header=True, inferSchema="true")
# df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2018/", sep=r'\t', header=True, inferSchema="true")
# df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2019/", sep=r'\t', header=True, inferSchema="true")

In [3]:
inputGeoHash = "dnd"

In [4]:
from math import log10

#  Note: the alphabet in geohash differs from the common base32 alphabet described in IETF's RFC 4648
# (http://tools.ietf.org/html/rfc4648)

__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
__decodemap = dict()
for i in range(len(__base32)):
    __decodemap[__base32[i]] = i


def decode_exactly(geohash):
    """
    Decode the geohash to its exact values, including the error
    margins of the result.  Returns four float values: latitude,
    longitude, the plus/minus error for latitude (as a positive
    number) and the plus/minus error for longitude (as a positive
    number).
    """
    lat_interval, lon_interval = (-90.0, 90.0), (-180.0, 180.0)
    lat_err, lon_err = 90.0, 180.0
    is_even = True
    for c in geohash:
        cd = __decodemap[c]
        for mask in [16, 8, 4, 2, 1]:
            if is_even: # adds longitude info
                lon_err /= 2
                if cd & mask:
                    lon_interval = ((lon_interval[0]+lon_interval[1])/2, lon_interval[1])
                else:
                    lon_interval = (lon_interval[0], (lon_interval[0]+lon_interval[1])/2)
            else:      # adds latitude info
                lat_err /= 2
                if cd & mask:
                    lat_interval = ((lat_interval[0]+lat_interval[1])/2, lat_interval[1])
                else:
                    lat_interval = (lat_interval[0], (lat_interval[0]+lat_interval[1])/2)
            is_even = not is_even
    lat = (lat_interval[0] + lat_interval[1]) / 2
    lon = (lon_interval[0] + lon_interval[1]) / 2
    return lat, lon, lat_err, lon_err


def decode(geohash):
    """
    Decode geohash, returning two float with latitude and longitude
    containing only relevant digits and with trailing zeroes removed.
    """
    lat, lon, lat_err, lon_err = decode_exactly(geohash)
    # Format to the number of decimals that are known
    lats = "%.*f" % (max(1, int(round(-log10(lat_err)))) - 1, lat)
    lons = "%.*f" % (max(1, int(round(-log10(lon_err)))) - 1, lon)
    if '.' in lats: lats = lats.rstrip('0')
    if '.' in lons: lons = lons.rstrip('0')
    return float(lats), float(lons)


def encode(latitude, longitude, precision=12):
    """
    Encode a position given in float arguments latitude, longitude to
    a geohash which will have the character count precision.
    """
    lat_interval = (-90.0, 90.0)
    lon_interval = (-180.0, 180.0)
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if longitude > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if latitude > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash += __base32[ch]
            bit = 0
            ch = 0
    return ''.join(geohash)

In [5]:
def convertToMonth(value):
    return time.strftime('%m', time.localtime(int(value)/1000))

udfConvertToMonth = F.udf(convertToMonth, StringType())

monthDf = df.withColumn("1_time", udfConvertToMonth("1_time"))
monthDf.limit(2).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface
0,01,27.764760,-73.670418,6.0,null,16526.799,102252.0,17187.525,6.0,0.0,296.64557,208.29962,null,0.0,0.0,24227.295,0.0,null
1,01,39.376466,-125.097760,6.0,null,22126.799,101660.0,19787.525,2.0,0.0,286.77057,210.79962,null,0.0,0.0,24227.295,0.0,null


In [6]:
def convertToGeoHash(lat, lon):
    return encode(lat, lon,3)

udfConvertToGeoHash = F.udf(convertToGeoHash, StringType())

geohashDf = df.withColumn("5_hash", udfConvertToGeoHash("2_lat", "3_lon"))

geohashDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,1422716400000,27.764760,-73.670418,6.00,null,16526.799,102252.0,17187.525,6.0,0.0000,296.64557,208.29962,null,0.0,0.0,24227.295,0.00000,null,dkg
1,1422716400000,39.376466,-125.097760,6.00,null,22126.799,101660.0,19787.525,2.0,0.0000,286.77057,210.79962,null,0.0,0.0,24227.295,0.00000,null,9pp
2,1422716400000,23.580932,-120.938083,6.00,null,23726.799,101649.0,10587.525,1.0,0.0000,294.27057,200.67462,null,0.0,0.0,24227.295,0.00000,null,9k1
3,1422716400000,59.622597,-123.937086,30.25,null,23726.799,99260.0,32587.525,99.0,0.3966,243.14555,222.67462,null,0.0,11.5,24227.295,0.06625,null,c4x
4,1422716400000,33.498223,-74.429542,6.00,null,24926.799,102410.0,22587.525,94.0,0.0000,294.64557,215.92462,null,0.0,0.0,24227.295,0.00000,null,dmg


In [7]:
filteredGeoHash = geohashDf.filter(geohashDf["5_hash"] == inputGeoHash)
filteredGeoHash.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,1422716400000,36.912775,-85.798101,14.75,null,20726.799,100005.0,19987.525,66.0,0.0,276.77057,208.79962,null,0.0,30.5,24227.295,0.08375,null,dnd
1,1422716400000,36.827081,-86.208368,15.00,null,20726.799,100635.0,19987.525,67.0,0.0,276.77057,209.04962,null,0.0,30.0,24227.295,0.08375,null,dnd
2,1422716400000,37.702586,-86.542842,14.50,null,20726.799,100819.0,20187.525,68.0,0.0,276.02057,208.92462,null,0.0,23.5,24227.295,0.08375,null,dnd
3,1422716400000,36.746945,-86.751663,15.00,null,20926.799,100645.0,19987.525,71.0,0.0,276.89557,209.29962,null,0.0,28.5,24227.295,0.08375,null,dnd
4,1422716400000,36.658890,-87.160538,15.00,null,20926.799,100936.0,19987.525,71.0,0.0,277.02057,209.67462,null,0.0,28.0,24227.295,0.08375,null,dnd


In [8]:
finalDf = filteredGeoHash.groupBy("1_time").agg(F.max("temperature_surface"),F.min("temperature_surface"),F.avg("total_precipitation_surface_3_hour_accumulation"),F.avg("temperature_surface"))

finalDf.limit(13).toPandas()

,1_time,max(temperature_surface),min(temperature_surface),avg(total_precipitation_surface_3_hour_accumulation),avg(temperature_surface)
0,1430341200000,296.62330,293.87330,0.000000,294.977467
1,1445396400000,280.52850,278.65350,0.000000,279.307346
2,1447038000000,282.31354,274.06354,0.000000,276.626040
3,1443236400000,290.90260,289.52760,1.442308,290.075677
4,1442026800000,291.46582,286.59082,0.005682,290.227184
5,1442847600000,298.00000,296.00000,0.000000,297.178571
6,1431874800000,302.32275,298.07275,0.116071,300.215607
7,1430924400000,301.92456,295.92456,0.000000,299.538196
8,1444467600000,287.68700,283.06200,0.237500,285.387000
9,1427749200000,292.63990,290.63990,0.000000,291.814900


In [10]:
finalDf.repartition(1).write.option("delimiter", "\t").csv("hdfs://orion11:11101/output/01")
print("done")

done
